In [ ]:
## Project -- Pruning on simple_vgg from LabGym
## Part 1: Data preprocessing & Prepruning training

In [ ]:
## Installation

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
import cv2
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from tensorflow.keras.optimizers import SGD
from google.colab import files
from tensorflow import keras
from sklearn.metrics import accuracy_score

In [ ]:
## Connect GoogleDrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
## Training data path

In [ ]:
train_ds_path = '/content/drive/MyDrive/medium_train'

In [ ]:
## Define the function for data preprocessing

In [ ]:

def preprocess_video(video_path, target_shape=(224, 224)):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, target_shape)
        frames.append(frame)
    cap.release()
    return np.array(frames)




In [ ]:
## Define the function for loading and preprocessing a folder of videos

In [ ]:

def load_and_preprocess_videos(folder_path, target_shape=(224, 224)):
    video_list = []
    labels = []
    for behavior_folder in os.listdir(folder_path):
        behavior_path = os.path.join(folder_path, behavior_folder)
        if not os.path.isdir(behavior_path):
            continue
        label = behavior_folder
        for filename in os.listdir(behavior_path):
            if filename.endswith(".avi"):
                video_path = os.path.join(behavior_path, filename)
                frames = preprocess_video(video_path, target_shape)
                video_list.append(frames)
                labels.append(label)
    return np.array(video_list), np.array(labels)


In [ ]:
# Define simple_vgg model

In [ ]:
def simple_vgg(inputs, filters, classes=3, level=2, with_classifier=False):
    if level < 2:
        layers = [2]
    elif level < 3:
        layers = [2, 3]
    elif level < 4:
        layers = [2, 3, 4]
    else:
        layers = [2, 3, 4, 4]

    x = inputs
    for i in layers:
        for n in range(i):
            if n == 0:
                if layers.index(i) == 0:
                    x = Conv2D(filters, kernel_size=(3, 3), padding='same', activation='relu')(x)
                    x = BatchNormalization()(x)
                else:
                    x = Conv2D(filters, kernel_size=(3, 3), padding='same', activation='relu')(x)
                    x = BatchNormalization()(x)
            else:
                x = Conv2D(filters, kernel_size=(3, 3), padding='same', activation='relu')(x)
                x = BatchNormalization()(x)
        x = MaxPooling2D(pool_size=(2, 2))(x)
        filters = int(filters * 2)

    x = Flatten()(x)

    if not with_classifier:
        return x
    else:
        x = Dense(int(filters / 2), activation='relu')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.5)(x)
        if classes == 2:
            x = Dense(1, activation='sigmoid')(x)
        else:
            x = Dense(classes, activation='softmax')(x)

        model = Model(inputs=inputs, outputs=x)
        return model


In [ ]:
# Define Hyperparameters

In [ ]:
input_shape = (224, 224, 3)
inputs = Input(shape=input_shape)
epochs = 10
batch_size = 32

In [ ]:
# Create the model

In [ ]:
model = simple_vgg(inputs, filters=64, classes=3, level=2, with_classifier=True)

In [ ]:
# Display model summary

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 batch_normalization (Batch  (None, 224, 224, 64)      256       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 batch_normalization_1 (Bat  (None, 224, 224, 64)      256       
 chNormalization)                                                
                                                                 
 max_pooling2d (MaxPooling2  (None, 112, 112, 64)      0     

In [ ]:
## Load and precess training data

In [ ]:
x, y = load_and_preprocess_videos(train_ds_path, target_shape=input_shape[:2])

In [ ]:
print(x.shape[1])


y = np.repeat(y, x.shape[1])

x = x.reshape(-1, 224, 224, 3)

print(x.shape)
print(y.shape)

16
(12720, 224, 224, 3)
(12720,)


In [ ]:
# Encode behavior labels into numerical values

In [ ]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
print(y_encoded)

[1 1 1 ... 2 2 2]


In [ ]:
# Split the data into training and validation sets

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x, y_encoded, test_size=0.2, random_state=42)

In [ ]:
# Set up callbacks

In [ ]:
model_path="/content/drive/MyDrive/model"

In [ ]:
cp=ModelCheckpoint(model_path,monitor='val_loss',verbose=1,save_best_only=True,save_weights_only=False,mode='min',save_freq='epoch')
es=EarlyStopping(monitor='val_loss',min_delta=0.001,mode='min',verbose=1,patience=4,restore_best_weights=True)
rl=ReduceLROnPlateau(monitor='val_loss',min_delta=0.001,factor=0.2,patience=2,verbose=1,mode='min',min_learning_rate=1e-7)

In [ ]:
# Train the model using the loaded and preprocessed data

In [ ]:
model.compile(optimizer=SGD(learning_rate=1e-4,momentum=0.9), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_val, y_val), callbacks=[cp,es,rl])

Epoch 1/10
318/318 [==============================] - ETA: 0s - loss: 0.5928 - accuracy: 0.7801
Epoch 1: val_loss improved from inf to 0.41354, saving model to /content/drive/MyDrive/model
318/318 [==============================] - 181s 506ms/step - loss: 0.5928 - accuracy: 0.7801 - val_loss: 0.4135 - val_accuracy: 0.8314 - lr: 1.0000e-04
Epoch 2/10
318/318 [==============================] - ETA: 0s - loss: 0.2355 - accuracy: 0.9140
Epoch 2: val_loss improved from 0.41354 to 0.19226, saving model to /content/drive/MyDrive/model
318/318 [==============================] - 168s 530ms/step - loss: 0.2355 - accuracy: 0.9140 - val_loss: 0.1923 - val_accuracy: 0.9336 - lr: 1.0000e-04
Epoch 3/10
318/318 [==============================] - ETA: 0s - loss: 0.1517 - accuracy: 0.9514
Epoch 3: val_loss improved from 0.19226 to 0.14622, saving model to /content/drive/MyDrive/model
318/318 [==============================] - 165s 519ms/step - loss: 0.1517 - accuracy: 0.9514 - val_loss: 0.1462 - val_acc

In [ ]:
# Clear session (effect not notieable)
tf.keras.backend.clear_session()

In [ ]:
# Save the trained model

In [ ]:
#model.save('3classification_model.h5')

In [ ]:
model.save('3c_trained_model_simpleVgg.keras')
## Save directly to googledrive
# model.save('/content/drive/MyDrive/model_path/3c_trained_model_simpleVgg.keras')


In [ ]:
## Download the model from colab

In [ ]:
files.download('3c_trained_model_simpleVgg.keras')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
## Load Test Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
test_ds_path = '/content/drive/MyDrive/small_test'
train_ds_path = '/content/drive/MyDrive/medium_train'

In [ ]:
input_shape = (224, 224, 3)
xt, yt = load_and_preprocess_videos(test_ds_path, target_shape=input_shape[:2])


In [ ]:
print("xt", xt.shape)
print("yt", yt.shape)

xt (30, 15, 224, 224, 3)
yt (30,)


In [ ]:
print(xt.shape[1])
yt = np.repeat(yt, xt.shape[1])
xt = xt.reshape(-1, 224, 224, 3)



15


In [ ]:
print("xt", xt.shape)
print("yt", yt.shape)

xt (450, 224, 224, 3)
yt (450,)


In [ ]:
print(yt)

['crawling' 'crawling' 'crawling' 'crawling' 'crawling' 'crawling'
 'crawling' 'crawling' 'crawling' 'crawling' 'crawling' 'crawling'
 'crawling' 'crawling' 'crawling' 'crawling' 'crawling' 'crawling'
 'crawling' 'crawling' 'crawling' 'crawling' 'crawling' 'crawling'
 'crawling' 'crawling' 'crawling' 'crawling' 'crawling' 'crawling'
 'crawling' 'crawling' 'crawling' 'crawling' 'crawling' 'crawling'
 'crawling' 'crawling' 'crawling' 'crawling' 'crawling' 'crawling'
 'crawling' 'crawling' 'crawling' 'crawling' 'crawling' 'crawling'
 'crawling' 'crawling' 'crawling' 'crawling' 'crawling' 'crawling'
 'crawling' 'crawling' 'crawling' 'crawling' 'crawling' 'crawling'
 'crawling' 'crawling' 'crawling' 'crawling' 'crawling' 'crawling'
 'crawling' 'crawling' 'crawling' 'crawling' 'crawling' 'crawling'
 'crawling' 'crawling' 'crawling' 'crawling' 'crawling' 'crawling'
 'crawling' 'crawling' 'crawling' 'crawling' 'crawling' 'crawling'
 'crawling' 'crawling' 'crawling' 'crawling' 'crawling' 'crawl

In [ ]:
## Load model

In [ ]:
model_path = '/content/drive/MyDrive/trained_model/3c_trained_model_simpleVgg.keras'

In [ ]:
model = keras.models.load_model(model_path)


In [ ]:
## Predict

In [ ]:
predictions = model.predict(xt)

15/15 [==============================] - 143s 9s/step


In [ ]:
print(predictions)

[[9.9945444e-01 3.6269674e-04 1.8292539e-04]
 [9.9964392e-01 1.3165106e-04 2.2437955e-04]
 [9.9883908e-01 6.6141227e-05 1.0949029e-03]
 ...
 [1.7919858e-03 9.9752885e-01 6.7908457e-04]
 [3.3291306e-03 9.9559826e-01 1.0727259e-03]
 [8.5180411e-03 9.8664480e-01 4.8371493e-03]]


In [ ]:
## Create true label

In [ ]:
label_encoder = LabelEncoder()
true_label_encoded = label_encoder.fit_transform(yt)

In [ ]:
print(true_label_encoded)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1]


In [ ]:
## Model accuracy

In [ ]:
accuracy = accuracy_score(true_label_encoded, predictions.argmax(axis=1))
rounded_accuracy = round(accuracy, 2)
print(f"Accuracy: {accuracy}%")

Accuracy: 0.7711111111111111
